In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip install scalecast

import pandas as pd
import numpy as np
from scalecast.Forecaster import Forecaster
from scalecast.auxmodels import auto_arima
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')

!pip install sktime
!pip install pmdarima


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# season = 'SpringSummer'
# ind = ''
# mode = 'VolTime_'
# Vol = pd.read_csv('/content/gdrive/MyDrive/ARCTICseaForecasting/PIOMAS/ICEF7d_Volsequences_' + season + '.csv',header=None)
# Vol.iloc[:,:] = Vol.iloc[:,:]*1000
# TS =  pd.read_csv('/content/gdrive/MyDrive/ARCTICseaForecasting/PIOMAS/ICEF7d_TSsequences_' + season + '.csv',header=None)
# cols = TS.columns[:]
# TS[cols] = TS[cols].apply(pd.to_datetime, errors='coerce')

In [ ]:
Vol = pd.read_csv('/content/gdrive/MyDrive/ARCTICseaForecasting/PIOMAS/IceTimeSeries.csv')
Vol['Volume'] = Vol['Volume']*1000 
Vol['Time'] = Vol['Time'].apply(pd.to_datetime, errors='coerce')
TS = Vol['Time']
Vol.set_index('Time', inplace=True)
Vol = Vol.drop(columns=['Thickness'])
Vol = Vol.asfreq('D')
Vol = Vol.fillna(0)
Vol# Vol = Vol['Volume']


,Volume
Time,
1979-01-01,26405.0
1979-01-02,26496.0
1979-01-03,26582.0
1979-01-04,26672.0
1979-01-05,26770.0
...,...
2022-06-26,14660.0
2022-06-27,14330.0
2022-06-28,13975.0


In [ ]:
# X_train, X_test = Vol.iloc[:TS.shape[0]-int(TS.shape[0]*20/100),:30], Vol.iloc[-int(TS.shape[0]*20/100):,:30]
# y_train, y_test = Vol.iloc[:TS.shape[0]-int(TS.shape[0]*20/100),:30], Vol.iloc[-int(TS.shape[0]*20/100):,:30]
y_train = Vol.iloc[:TS.shape[0]-int(TS.shape[0]*20/100),:]
y_test = Vol.iloc[-int(TS.shape[0]*20/100):,:]

In [ ]:
int(TS.shape[0]*20/100)

3175

In [ ]:
# # step 2: specifying forecasting horizon
# from sktime.forecasting.base import ForecastingHorizon
# fh = np.arange(1, 8)

# # step 3: specifying the forecasting algorithm
# from sktime.forecasting.naive import NaiveForecaster
# # forecaster = NaiveForecaster(strategy="last")
# # from sktime.forecasting.arima import AutoARIMA
# forecaster = AutoARIMA(suppress_warnings=True)

# # step 4: fitting the forecaster
# forecaster.fit(y_train, fh=fh)

# # step 5: querying predictions
# y_pred = forecaster.predict(fh)
# y_pred

In [ ]:
from sktime.forecasting.base import ForecastingHorizon
from sktime.forecasting.compose import make_reduction
from sklearn.svm import SVR

fh = ForecastingHorizon(y_test.index, is_relative=False)

regressor = SVR(kernel='sigmoid')
mdl = 'SVRsigmoid'
forecaster = make_reduction(regressor, window_length=30, strategy="recursive")
forecaster.fit(y_train)
y_pred = forecaster.predict(fh)
y_pred

In [ ]:
d = pd.DataFrame([])
d = y_test
np.savetxt('/content/gdrive/MyDrive/ARCTICseaForecasting/PIOMAS/export/BaselineModels_test_targetdata_VOLF.csv', d, delimiter=',')

dp = pd.DataFrame([])
dp = y_pred
np.savetxt('/content/gdrive/MyDrive/ARCTICseaForecasting/PIOMAS/export/BaselineModels_test_predsdata_VOLF_' + mdl +'.csv', dp, delimiter=',')


In [ ]:
from sktime.forecasting.base import ForecastingHorizon
from sktime.forecasting.compose import make_reduction
from sklearn.ensemble import GradientBoostingRegressor

fh = ForecastingHorizon(y_test.index, is_relative=False)

regressor = GradientBoostingRegressor()
mdl = 'GBDT'
forecaster = make_reduction(regressor, window_length=30, strategy="recursive")
forecaster.fit(y_train)
y_pred = forecaster.predict(fh)
y_pred

,Volume
2013-10-21,6275.876824
2013-10-22,6275.876824
2013-10-23,6275.876824
2013-10-24,6275.876824
2013-10-25,6275.876824
...,...
2022-06-26,6275.876824
2022-06-27,6275.876824
2022-06-28,6275.876824
2022-06-29,6275.876824


In [ ]:
d = pd.DataFrame([])
d = y_test
np.savetxt('/content/gdrive/MyDrive/ARCTICseaForecasting/PIOMAS/export/BaselineModels_test_targetdata_VOLF.csv', d, delimiter=',')

dp = pd.DataFrame([])
dp = y_pred
np.savetxt('/content/gdrive/MyDrive/ARCTICseaForecasting/PIOMAS/export/BaselineModels_test_predsdata_VOLF_' + mdl +'.csv', dp, delimiter=',')

In [ ]:
from sktime.forecasting.base import ForecastingHorizon
from sktime.forecasting.compose import make_reduction
from sklearn.ensemble import RandomForestRegressor

fh = ForecastingHorizon(y_test.index, is_relative=False)

regressor = RandomForestRegressor()
mdl = 'RF'
forecaster = make_reduction(regressor, window_length=30, strategy="recursive")
forecaster.fit(y_train)
y_pred = forecaster.predict(fh)
y_pred

,Volume
2013-10-21,7216.98
2013-10-22,7333.04
2013-10-23,7436.61
2013-10-24,7531.10
2013-10-25,7598.94
...,...
2022-06-26,24667.90
2022-06-27,24656.74
2022-06-28,24667.11
2022-06-29,24661.28


In [ ]:
d = pd.DataFrame([])
d = y_test
np.savetxt('/content/gdrive/MyDrive/ARCTICseaForecasting/PIOMAS/export/BaselineModels_test_targetdata_VOLF.csv', d, delimiter=',')

dp = pd.DataFrame([])
dp = y_pred
np.savetxt('/content/gdrive/MyDrive/ARCTICseaForecasting/PIOMAS/export/BaselineModels_test_predsdata_VOLF_' + mdl +'.csv', dp, delimiter=',')

In [ ]:
from sktime.forecasting.arima import AutoARIMA
from sktime.forecasting.base import ForecastingHorizon
from sktime.forecasting.compose import make_reduction

fh = ForecastingHorizon(y_test.index, is_relative=False)

mdl = 'ARIMA'
forecaster = AutoARIMA(sp=12, suppress_warnings=True) 
forecaster.fit(y_train)
y_pred = forecaster.predict(fh)
y_pred

,Volume
2013-10-21,6601.859136
2013-10-22,6643.801767
2013-10-23,6685.386061
2013-10-24,6726.615081
2013-10-25,6767.491862
...,...
2022-06-26,11511.175294
2022-06-27,11511.175294
2022-06-28,11511.175294
2022-06-29,11511.175294


In [ ]:
forecaster.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                12701
Model:               SARIMAX(1, 1, 5)   Log Likelihood              -99615.825
Date:                Thu, 25 Aug 2022   AIC                         199245.650
Time:                        13:49:46   BIC                         199297.796
Sample:                    01-01-1979   HQIC                        199263.093
                         - 10-09-2013                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.9915      0.002    604.681      0.000       0.988       0.995
ma.L1         -1.7930      0.002   -908.810      0.000      -1.797      -1.789
ma.L2          0.8052      0.003    268.063      0.000       0.799       0.811
ma.L3          0.0054      0.003      1.622      0.105      -0.001       0.012
ma.L4         -0.0182      0.004     -4.898      0.000      -0.025      -0.011
ma.L5          0.0245      0.003      9.745      0.000       0.020       0.029
sigma2      3.775e+05    312.061   1209.785      0.000    3.77e+05    3.78e+05
===================================================================================
Ljung-Box (L1) (Q):                   0.04   Jarque-Bera (JB):         568735246.38
Prob(Q):                              0.84   Prob(JB):                         0.00
Heteroskedasticity (H):               0.52   Skew:                           -28.26
Prob(H) (two-sided):                  0.00   Kurtosis:                      1038.17
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [ ]:
mdl = 'SARIMAX'
d = pd.DataFrame([])
d = y_test
np.savetxt('/content/gdrive/MyDrive/ARCTICseaForecasting/PIOMAS/export/BaselineModels_test_targetdata_VOLF.csv', d, delimiter=',')

dp = pd.DataFrame([])
dp = y_pred
np.savetxt('/content/gdrive/MyDrive/ARCTICseaForecasting/PIOMAS/export/BaselineModels_test_predsdata_VOLF_' + mdl +'.csv', dp, delimiter=',')